In [1]:
import mediapipe as mp
import cv2

In [2]:
mp_drawing  = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## Make Detection

In [19]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) ,         mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) ,          mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) ,         mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) ,        mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Capture the Landmarks

In [3]:
import csv
import numpy as np
import os

In [10]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)

In [14]:
landmarks = ['class']
for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val) ,'y{}'.format(val),'z{}'.format(val),'v{}'.format(val) ]

In [16]:
with open('coords.csv' , mode='w',newline='') as f:
    csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [23]:
class_name = 'Victouris'

In [24]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) ,         mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) ,          mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) ,         mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) ,        mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))

        # export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row
            row.insert(0 , class_name)

            with open('coords.csv' , mode='a',newline='') as f:
                csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Create Model

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
df = pd.read_csv('coords.csv')

In [28]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.500657,0.613720,-1.115281,0.999839,0.532508,0.543845,-1.033326,0.999699,0.555143,...,-0.001737,0.0,0.566240,0.532601,0.025169,0.0,0.572219,0.527062,0.026222,0.0
1,Happy,0.500474,0.614542,-1.168532,0.999805,0.532203,0.544535,-1.082736,0.999641,0.554549,...,-0.001005,0.0,0.565989,0.533436,0.025574,0.0,0.571853,0.527762,0.026638,0.0
2,Happy,0.500569,0.615019,-1.205891,0.999765,0.532187,0.544943,-1.118739,0.999578,0.554404,...,-0.002798,0.0,0.565154,0.528870,0.023352,0.0,0.570988,0.523350,0.024342,0.0
3,Happy,0.500593,0.615359,-1.234285,0.999719,0.532097,0.545353,-1.146037,0.999506,0.554169,...,-0.000731,0.0,0.563805,0.531685,0.026234,0.0,0.569705,0.525997,0.027333,0.0
4,Happy,0.500079,0.616703,-1.253217,0.999695,0.531705,0.546454,-1.167382,0.999467,0.553767,...,-0.001890,0.0,0.563329,0.532082,0.023685,0.0,0.569285,0.526535,0.024638,0.0


In [29]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
341,Victouris,0.457547,0.646542,-1.044208,0.999936,0.481424,0.589085,-0.951438,0.999869,0.496858,...,-0.011553,0.0,0.517103,0.563691,0.006568,0.0,0.521665,0.559760,0.006955,0.0
342,Victouris,0.466308,0.636127,-1.051559,0.999935,0.488479,0.581103,-0.953093,0.999868,0.505310,...,-0.010608,0.0,0.529222,0.554652,0.009933,0.0,0.533759,0.550489,0.010437,0.0
343,Victouris,0.475955,0.634087,-1.085917,0.999935,0.496976,0.578406,-0.986342,0.999868,0.515373,...,-0.010714,0.0,0.534646,0.552598,0.010855,0.0,0.538995,0.548620,0.011413,0.0
344,Victouris,0.486417,0.628515,-1.067564,0.999939,0.509187,0.571131,-0.972238,0.999876,0.527607,...,-0.010303,0.0,0.547147,0.543724,0.012147,0.0,0.551598,0.539747,0.012823,0.0
345,Victouris,0.494866,0.624417,-1.084342,0.999941,0.519067,0.566119,-0.994680,0.999882,0.537186,...,-0.008745,0.0,0.558168,0.533421,0.016059,0.0,0.562657,0.529334,0.016943,0.0


In [30]:
x = df.iloc[: , 1:]
y = df.iloc[: , 0]

In [33]:
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state=0)

In [34]:
len(X_train) , len(X_test)

(276, 70)

## Model Preparation

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [37]:
pipelines = {
    'lr':make_pipeline(StandardScaler() , LogisticRegression()),
    'rc':make_pipeline(StandardScaler() , RidgeClassifier()),
    'rf':make_pipeline(StandardScaler() , RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler() , GradientBoostingClassifier())
}

In [43]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [45]:
fit_models = {}
for algo , pipeline in pipelines.items():
    model = pipeline.fit(X_train , y_train)
    fit_models[algo] = model

## Evaluate Model

In [46]:
from sklearn.metrics import accuracy_score
import pickle

In [47]:
for algo , model in fit_models.items():
    pred = model.predict(X_test)
    print(algo , accuracy_score(y_test , pred))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [48]:
# save the model
with open('body_language.pkl' , 'wb') as f:
    pickle.dump(fit_models['rf'], f)


## Make Predictions ( Real Time )

In [49]:
with open('body_language.pkl','rb') as f:
    model = pickle.load(f)

In [50]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) ,         mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) ,          mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) ,         mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) ,        mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))

        # export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class , body_language_prob)

            # grab ear coords
            coords = tuple(
                np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x ,                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)) , 
                        [640,480]).astype(int)
                        )

            cv2.rectangle(image,
            (coords[0],coords[1]+5) , 
            (coords[0]+len(body_language_class)*20,coords[1]-30), (245 , 117 , 16),-1)

            cv2.putText(image , body_language_class , coords ,
             cv2.FONT_HERSHEY_SIMPLEX , 1 , (255,255,255),2 , cv2.LINE_AA)

        except:
            pass
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()